<h1><center>CMSC320: Intro to Data Science Tutorial</center></h1>

In [1]:
import pandas as pd
import numpy as np

In [8]:
energy_data = pd.read_excel("Table_2.1_Energy_Consumption_by_Sector-2.xlsx", sheetname="Monthly Data", header=10, skiprows=[11])
print(energy_data.head(5))

       Month  Primary Energy Consumed by the Residential Sector  \
0 1973-01-01                                           1313.816   
1 1973-02-01                                           1150.011   
2 1973-03-01                                            970.362   
3 1973-04-01                                            709.631   
4 1973-05-01                                            544.596   

   Total Energy Consumed by the Residential Sector  \
0                                         1932.187   
1                                         1687.255   
2                                         1497.067   
3                                         1177.661   
4                                         1015.008   

   Primary Energy Consumed by the Commercial Sector  \
0                                           656.607   
1                                           621.766   
2                                           490.154   
3                                           362.003 